In [1]:
import os
from pyciemss.PetriNetODE.interfaces import (
    load_and_sample_petri_model,
    load_and_calibrate_and_sample_petri_model
)
from pyciemss.visuals import plots

In [2]:
DEMO_PATH = "../../notebook/integration_demo/"
ASKENET_PATH = "https://raw.githubusercontent.com/DARPA-ASKEM/Model-Representations/main/petrinet/examples/sir_typed.json"

## load_and_sample_petri_model

In [3]:
num_samples = 3
timepoints = [0.5, 1.0, 2.0, 3.0, 4.0]

# Run sampling w/o an intervention
samples = load_and_sample_petri_model(ASKENET_PATH,
                                      num_samples,
                                      timepoints=timepoints,
                                      )

# Save results
samples.to_csv(os.path.join(DEMO_PATH, "results_petri/sample_results.csv"), index=False)

In [4]:
interventions = [(1.1, "beta", 1.0), (2.1, "gamma", 0.1), (1.3, "beta", 2.0), (1.4, "gamma", 0.3)]

result = load_and_sample_petri_model(
    ASKENET_PATH, num_samples, timepoints=timepoints, interventions=interventions,
    time_unit="days",
    visual_options={"title": "SIR Model"}
)

# Save results
result["data"].to_csv(
    os.path.join(DEMO_PATH, "results_petri/sample_results_w_interventions.csv"),
    index=False,
)

In [11]:
plots.ipy_display(result["visual"])

## load_and_calibrate_and_sample_petri_model

In [6]:
data_path = os.path.join(DEMO_PATH, "data.csv")
num_samples = 100
timepoints = [0.0, 1.0, 2.0, 3.0, 4.0]

# Run the calibration and sampling
result = load_and_calibrate_and_sample_petri_model(
    ASKENET_PATH,
    data_path,
    num_samples,
    timepoints=timepoints,
    verbose=True,
    visual_options={"title": "Intervened SIR Model"}
)

# Save results
result['data'].to_csv(
    os.path.join(DEMO_PATH, "results_petri/calibrated_sample_results.csv"), index=False
)

iteration 0: loss = 62.79788780212402
iteration 25: loss = 60.172391176223755
iteration 50: loss = 58.51065230369568
iteration 75: loss = 57.90844249725342
iteration 100: loss = 57.26304745674133
iteration 125: loss = 57.494603872299194
iteration 150: loss = 57.651503562927246
iteration 175: loss = 56.44858956336975
iteration 200: loss = 56.126389503479004
iteration 225: loss = 57.39357018470764
iteration 250: loss = 56.28469395637512
iteration 275: loss = 56.06474947929382
iteration 300: loss = 57.09253716468811
iteration 325: loss = 56.88748478889465
iteration 350: loss = 56.51016354560852
iteration 375: loss = 56.57038903236389
iteration 400: loss = 56.5693793296814
iteration 425: loss = 56.80248999595642
iteration 450: loss = 56.69068431854248
iteration 475: loss = 56.573203802108765
iteration 500: loss = 56.907164335250854
iteration 525: loss = 56.58754873275757
iteration 550: loss = 56.64813804626465
iteration 575: loss = 56.54149413108826
iteration 600: loss = 56.77300000190735


In [10]:
plots.ipy_display(result["visual"])